# A streamlined topsApp.py notebook with plotting

**Authors**: Gareth Funning, based on original notebooks by David Bekaert, Heresh Fattahi, Piyush Agram, Eric Fielding and Scott Henderson


**topsApp.py is a pair-by-pair interferometric processor that takes as input two Sentinel-1 SAR acquisitions acquired in TOPS mode.** 

topsApp.py will not work for other Sentinel-1 acquisition formats such as Stripmap. ISCE2's stripmapApp.py supports interferometric stripmap processing of Sentinel-1 and other sensors. At this time, ISCE2 topsApp only supports SLC data from Sentinel-1 A and B. Processing is supported across the Sentinel-1 constellation, i.e. data acquired from A and B (and future C, D) can be combined.

To illustrate the usage of topsApp.py, we will use a Sentinel-1 dataset capturing the surface deformation as result of the 15 May 2020 Mw6.5 Monte Cristo Range Earthquake that occurred in Nevada [(details here)](https://en.wikipedia.org/wiki/2020_Nevada_earthquake). The exercise runs the workflow step by step to generate an interferogram!

## 0. Initial setup of the notebook

In [ ]:
# Configure ISCE2 python library
# https://github.com/isce-framework/isce2/issues/258
import isce
import logging
import os 
import asf_search      
import matplotlib.pyplot as plt    
import xarray as xr
import customPlots as cp

root_logger = logging.getLogger()
root_logger.setLevel('WARNING')

# Set Environment variables so that you can call ISCE2 Apps from the command line
os.environ['ISCE_HOME'] = os.path.dirname(isce.__file__)
os.environ['PATH']+='{ISCE_HOME}/bin:{ISCE_HOME}/applications'.format(**os.environ)

print('ISCE2 Version:', isce.__version__)

%matplotlib inline
%config InlineBackend.figure_format='retina'

Make some directories in which to store your data and run your things:

In [ ]:
tutorial_home_dir = os.getcwd()
print("Notebook directory: ", tutorial_home_dir)

# directory for data downloads
slc_dir = os.path.join(tutorial_home_dir,'slcs')
orbit_dir = os.path.join(tutorial_home_dir,'orbits')
insar_dir = tutorial_home_dir

# generate all the folders in case they do not exist yet
os.makedirs(slc_dir, exist_ok=True)
os.makedirs(orbit_dir, exist_ok=True)
os.makedirs(insar_dir, exist_ok=True)

## 1. Data download and setup

### 1.1 Downloading SLCs

The ASF vertex page (https://search.asf.alaska.edu/) offers a GUI to visually search for available Sentinel-1 data over your area of interest. Once you have found your data, you can download it from the GUI, or export a Python download script (usually my favored way). To download the Sentinel-1 data from ASF, you must have a NASA Earthdata account and have that configured in your `$HOME/.netrc` file. 

Another option (shown below) is to use ASF's Python Search Client (https://github.com/asfadmin/Discovery-asf_search) to download two SLCs $-$ give the full granule names (the file names) for the files you want:

In [ ]:
# Reads your ~/.netrc file
session = asf_search.ASFSession()

reference = 'S1A_IW_SLC__1SDV_20200511T135117_20200511T135144_032518_03C421_7768'
secondary = 'S1B_IW_SLC__1SDV_20200517T135026_20200517T135056_021622_0290CB_99E2'
granules = [reference, secondary]

results = asf_search.granule_search(granules)
results.download(path=slc_dir, processes=2, session=session)

Files are usually 4+GB in size each. You can check them by executing the command below:

In [ ]:
!ls -lh {slc_dir}

### 1.2 The SLC filenaming convention

TOPS SLC product files delivered from ESA are zip archives. When unpacked the zip extension will be replaced by SAFE. The products are therefore also frequently called SAFE files. topsApp.py can read the data from either a zip file or a SAFE file. To limit disk usage, it is recommended to not unzip the individual files.

The zip or SAFE filenames provide information on the product type, the polarization, and the start and stop acquisition time. For example: S1A_IW_SLC__1SDV_20200511T135117_20200511T135144_032518_03C421_7768.zip
- Type = slc
- Polarization = Dual polarization
- Date = 20200511
- UTC time of acquisition = ~13:51
- Sensing start for the acquisition was 20200511 at 13:51:17

### 1.3 Downloading orbits

In addition to the **SAFE files**, **orbit files** and the **auxiliary instrument files** are required for ISCE processing. The Copernicus Scihub and ESA servers discontinued service in October 2023 in favor of the new Copernicus Data Space Ecosystem. The new service no longer allows anonymous public downloads (using the gnssuser), which means you must register for either a Dataspace account (to use the CDSE data) or a NASA Earthdata account (to use the orbits provided by ASF). The NASA EarthData account for the ASF Sentinel-1 orbits is the same as for Sentinel-1 data download.

Although Sentinel-1 restituted orbits (RESORB) are of good quality, it is recommended to use the precise orbits (POEORB) when available. Typically, precise orbits are available with a 15 to 20-day lag from the day of the acquisition. 

In the next cell, we use a new Python program `eof` that was written by a JPL Radar engineer, Scott Staniewicz, to download orbits from either the new Copernicus DataSpace server or the ASF copy of the Copernicus orbit data. The advantage of the ASF option is that it uses the same EarthData login as the Sentinel-1 SLC download, so we force the ASF option. The `eof` program will skip the download if the orbit data is already available.
See https://github.com/scottstanie/sentineleof for more details.

In [ ]:
# if you don't already have eof installed, run this:
!pip install sentineleof

In [ ]:
# if it is installed, run eof!
!eof --search-path {slc_dir} --save-dir {orbit_dir} --force-asf

## 2. Configuring inputs to `topsApp.py`

In ISCE2, the input variables to topsApp.py are controlled through an app xml file. All apps in ISCE have example xml files included in the ISCE distribution. You can find these under [**examples/input_files**](https://github.com/isce-framework/isce2/tree/main/examples/input_files) on the github repo for ISCE2. For convenience, I have included an example *topsApp.xml* file in this directory. 

In [ ]:
# use a simple shell command to view the file
!cat topsApp_example.xml

**I strongly suggest** that you make a copy of this file, call it `topsApp.xml` and edit it yourself, either in the Jupyter text editor, or with a text editor such as `gedit`, `nano` or `vi`. 

In our example of a `topsApp.xml` file, only a small number of the possible input variables are set $-$ the most important ones. The processing will happily run with those on their own. If you are curious what other possibilities are out there, you can ask `topsApp.py` for a full list:

In [ ]:
!topsApp.py --help

## 3. `topsApp.py` processing steps

The topsApp.py workflow can be called with a single command-line call to topsApp.py; by default it will run all the required processing steps with inputs pulled from the topsApp.xml file. Although this is an attractive feature, it is recommended to run topsApp.py with “steps” enabled. This will allow you to re-start the processing from a given processing step. If “steps” are not used, users must restart processing from the beginning of the workflow after fixing any downstream issues with the processing. 

The "--help" switch lists all the steps involved in the processing:

In [ ]:
!topsApp.py --help --steps

### 3.1 Processing your interferogram in one go

If all the inputs are set up properly and in the right places, you can run all of `topsApp.py` in one go, with the command below:

(note we are directing all of the output to a log file to keep things clean)

In [ ]:
# this assumes you have a 'topsApp.xml' file set up in the processing directory
os.chdir(insar_dir)
!topsApp.py --steps &> topsApp.log

If you did this right, the code cell above will take a while to run. If you didn't, then `topsApp.py` will probably crash pretty quickly. If that is the case, you might get some insight as to why by running:

In [ ]:
!tail -40 topsApp.log

### 3.2 Common ways for `topsApp.py` to crash

1) **You don't have a `topsApp.xml` file**

If you run `topsApp.py` without explicitly naming the xml file that contains all of the configuration and file information, it assumes your file is called `topsApp.xml`. If that doesn't exist, you are not going to get very far.

2) **Your orbit files are missing or wrong**

If topsApp can't find appropriate orbit files for your data, it won't know where your SLCs are located in space, and it will probably fail to find any overlap between them, muttering comments about 'bboxes' (bounding boxes, i.e. the extents of the images) as it does so.

3) **Your SLCs don't overlap with each other (or with your bounding box, if you are using one)**

If your SLCs legitimately don't overlap, topsApp will crash. If you specify a bounding box (which is an option) and your SLCs don't overlap with *that*, topsApp will crash.

4) **It has trouble downloading a digital elevation model**

A DEM is essential for geocoding and correcting data for topographic effects. If you are unable to download one (which is an early step in the `topsApp.py`, then something about your setup is wrong $-$ or, you are in a location where the standard DEM is not available (the SRTM DEM that ISCE typically uses is not available north of 60$^\circ$N or south of 56$^\circ$S. In any case, your processing will be hosed.

5) **You have a lot of decorrelation in your data, and it crashes at the ESD step**

Enhanced Spectral Diversity, or ESD, is a way of using the diversity of Doppler shifts in the data to estimate a fine, along-track alignment between your SLCs. This will fail if you have too much decorrelation in your data. If that is the case, try to restrict your processing to areas that have less decorrelation, or switch ESD off in your `topsApp.xml` file.

## 4. Plotting the output

Assuming your processing got somewhere, then you can try these! 

The final interferogram files from `topsApp.py` are stored in the 'merged' directory. See what you have there by running this command:

In [ ]:
!ls -l merged

### 4.1 The complex interferogram in radar coordinates

If you stop the processing mid-stream, perhaps before unwrapping, you can look at your unfiltered first interferogram using the `customPlots.py` plotting script supplied with this notebook:

In [ ]:
cp.plotcomplexdata('merged/topophase.flat.vrt', 
                title="MERGED IFG ", aspect=3,
                datamin=0, datamax=10000, draw_colorbar=True)

### 4.2 The unwrapped interferogram and its connected components in radar coordinates

In [ ]:
cp.plotdata('merged/filt_topophase.unw', band=2,
         title="UNW FILT IFG [rad] ",
         colormap='jet', colorbar_orientation="vertical")

cp.plotdata('merged/filt_topophase.unw.conncomp', band=1,
         title="UNW CONN COMP [-] ", colormap='jet',
         colorbar_orientation="vertical",
         nodata=0)

### 4.3 The geocoded interferograms

In [ ]:
cp.plotcomplexdata('merged/filt_topophase.flat.geo', 
                title="GEO FILT IFG [rad]",
                datamin=0, datamax=10000, draw_colorbar=True)

cp.plotdata('merged/filt_topophase.unw.geo', band=2,
         title="UNW GEO FILT IFG [rad] ", colormap='jet',
         colorbar_orientation="vertical", datamin=-20, datamax=50)


### 4.4. (optionally) The dense offsets

The output file, **dense_offsets.bil**, is a two-band file containing the azimuth and range offsets, placed in the "*merged*" folder . The correlation coefficient is contained in the **dense_offsets_snr.bil** file.

Here is a plot of the azimuth and range offsets:

In [ ]:
# This will only work if you enabled pixel offset generation
cp.plotdata('merged/filt_dense_offsets.bil.geo', band=2,
         title="DENSE RANGE OFFSETS [pixels] ",
         datamin=-0.1, datamax=0.1,
         colormap='jet', colorbar_orientation="vertical")
cp.plotdata('merged/filt_dense_offsets.bil.geo',band=1,
         title="DENSE AZIMUTH OFFSETS [pixels] ",
         datamin=-0.1, datamax=0.1,
         colormap='jet', colorbar_orientation="vertical")